[View in Colaboratory](https://colab.research.google.com/github/ThreeOne31/CNN_Bird_Species_Classification/blob/master/inception_v3.ipynb)

clone github data into collab 

In [0]:
!git clone https://github.com/MbeleLebohang-uct/birds-data.git

Download a pre trained model from google drive 

In [1]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#download file
file_id = '1GIJrqTFtbaorkUBSTrWF93bOmG_vHMoc' # URL id. 
downloaded = drive.CreateFile({'id': file_id})
name=downloaded['title']
print(name)
downloaded.GetContentFile('inception_v3_notop.h5')



inception_v3_notop.h5


CNN feature etxraction 

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Fri Aug 24 19:23:01 2018

@author: Khomo
"""

import os
#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
import numpy as np
#import seaborn as sns
#from PIL import Image

from tensorflow.python.keras import layers
from tensorflow.python.keras import Model

from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.applications.inception_v3 import InceptionV3
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
#from tensorflow.python.keras.preprocessing.image import array_to_img, img_to_array, load_img


#=========================================================================
#                           variables
#=========================================================================
#base_dir='workingDIR'        #train in selected images
base_dir ='birds-data/images' #traing on 200 classes
class_names = np.array(os.listdir(base_dir))

#-------------------------generate data--------------------------------
#Split training and validation data by 0.8 to 0.2
datagen = ImageDataGenerator(rescale=1./255) #,validation_split=0.2)

#fetch images from base_dir in batches of 32 images
#Returns (x,y) tuple
train_generator=datagen.flow_from_directory(
    base_dir,
    #subset='training',
    target_size=(150, 150),
    batch_size=32,
    shuffle=False,
    class_mode='sparse'
)

#========================================================================================
#               LOAD INCEPTION WEIGHTS
#========================================================================================

#Include top = False dont include classification layer
local_weights_file = 'inception_v3_notop.h5'
pre_trained_model = InceptionV3(input_shape=(150, 150, 3), include_top=False, weights=None)
pre_trained_model.load_weights(local_weights_file)

#extract features set trainable to False use the original wieghts
for layer in pre_trained_model.layers:
  layer.trainable = False

# Mixed7 feature extraction layer  of 7x7 no bottleneck 
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape:', last_layer.output_shape)
last_output = last_layer.output

# Flatten the output layer to 1 dimension
x_flat = layers.Flatten()(last_output)

# Configure and compile the model
model = Model(pre_trained_model.input, x_flat)
model.compile(loss='mean_squared_error',
              optimizer=Adam(lr=0.001),
              metrics=['acc'])


Found 11788 images belonging to 200 classes.
last layer output shape: (None, 7, 7, 768)


In [0]:
features_pretrained =model.predict_generator(train_generator)
#print(np.shape(features_pretrained))
#print(features_pretrained)

Function Definitions SVM part

In [4]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import seaborn as sns

import PIL.Image
import itertools
import pickle



from sklearn import cross_validation, grid_search
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn.externals import joblib

#---------------------variables---------------------
class_names = np.array(os.listdir(base_dir))

#========================================================================================
#           SVM Training Function 
#========================================================================================
def train_svm_classifer(features, labels, model_output_path):
  """
  train_svm_classifer will train a SVM, saved the trained and SVM model and
  report the classification performance

  features: array of input features
  labels: array of labels associated with the input features
  model_output_path: path for storing the trained svm model
  """
  
  # save 20% of data for performance evaluation
  X_train, X_test, y_train, y_test = cross_validation.train_test_split(features, labels, test_size=0.2)
  
  #choose a linear classifier
  clf = SVC(C=1, kernel='linear', probability=True,decision_function_shape='ovr')
  joblib.dump(clf, model_output_path)
  clf.fit(X_train, y_train)
  
  

  y_predict=clf.predict(X_test)
  
  cm=confusion_matrix(y_test, y_predict)
  cm_normalized = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]

  #print(cm)
  plot_confusion_matrix(cm_normalized,class_names)
  
  labels=sorted(list(set(labels)))
  print("\nConfusion matrix:")
  print("Labels: {0}\n".format(",".join(str(labels))))
  print(confusion_matrix(y_test, y_predict, labels= labels))

  print("\nClassification report:")
  print(classification_report(y_test, y_predict, target_names=class_names))
  
  
  
#=========================================================================================================
#                     Confusion Matrix Plot
#=========================================================================================================

def plot_confusion_matrix(cm, classes, cmap=plt.cm.Blues, title='Confusion matrix'):
  
  """
  This function prints and plots the confusion matrix.
  Normalization can be applied by setting `normalize=True`.
  """
  #cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


  plt.imshow(cm, cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks, classes)

  #fmt = '.2f' if normalize else 'd'
  thresh = cm.max() / 2.

  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      plt.text(j, i, format(cm[i, j], '.2f'),
               horizontalalignment="center",
               color="white" if cm[i, j] > thresh else "black")

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')

  

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Classification Pass: CNN Features into SVM

In [11]:
features =[] #array of features 
labels =[]   #array of labels

x_y =train_generator
for i in range(0,len(train_generator)):
  img_batch = x_y[i]
  batch_features=img_batch[0]  #32 distinct features 
  batch_labels=img_batch[1]  #32 distinct labes
  #print(batch_labels)
  #print(len(batch_labels))
  #break
  
  #get feature labels pair sure this will be done better in a dict
  for k in range(0,len(batch_features)):
    arry_batch_features=batch_features[k]
    flat_features =arry_batch_features.flatten()
    features.append(flat_features)
    #print(np.shape(flat_features))
    #print(arry_batch_features)
    labels.append(batch_labels[k])
    #print(batch_labels[k])
    
   

model_output_path='model/model.pkl'

train_svm_classifer(features_pretrained, labels, model_output_path)
   

IsADirectoryError: ignored

In [1]:
!ls

sample_data


Upload a trained CNN_SVM model to drive 

In [0]:
# Create & upload a file.
uploaded = drive.CreateFile({'title': 'model.pkl'})
uploaded.SetContentFile('model/model.pkl')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))